# Less Common Joins

Although INNER, LEFT, RIGHT, and FULL are the most common joins that we will see used within SQL Server, these do not account for all eventualities when we are writing our queries and so the SQL language provides several other methods of combining tables.

These are not all JOINs in the true sense of the word (combining tables in a side by side manner), but they are included in this section as they still combine table information in one way or another.

These are:

-   CROSS JOIN
-   UNION (ALL)
-   EXCEPT
-   INTERSECT

We will utilise the same sales.salesOrderHeaderSmall table for most of these demos but additionally use a slight variant of that table (it has the same structure just slightly different data) called sales.salesOrderHeaderSmall2:

In [ ]:
select *
from JupyterDatabase.sales.salesOrderHeaderSmall;

In [ ]:
select *
from JupyterDatabase.sales.salesOrderHeaderSmall2;

We will cover these each in turn and attempt to answer the following questions using these methods:

-   Query One
    -   I wish to see all orderIDs that have been placed without duplication
-   Query Two
    -   Which employee IDs have made a sale in both tables
-   Query Three
    -   Are there any Order Dates which are in sales.salesOrderHeaderSmall but not in sales.salesOrderHeaderSmall2

#### CROSS JOIN

This is a rarely seen join as generally there is no call for it in a production environment and mostly it is seen producing demo data or bloating datasets for performance tuning purposes.

However, you may well find a legitimate use for this (or need to create data yourself) and therefore it is useful to know.

In effect a CROSS JOIN creates a Cartesian Product by joining every record in your FROM table with every record in your JOIN table.

Here's how this looks in tabular form:

![CrossJoin](https://www.sqlworld.co.uk/images/Blogs/SQLServer/CrossJoin.jpg)

Because the purpose of the CROSS JOIN is to join every record it does not support the ON clause:

In [ ]:
select a.employeeID, b.employeeID
from JupyterDatabase.hr.employeeSmall as a
cross join JupyterDatabase.hr.employeeSmall as b;

#### UNION (ALL)

The UNION operator is a useful operator which can be used to combine multiple datasets into one large dataset.

It is best thought as stacking one dataset on top of another to make one single, large dataset.

As that implies, however, the datasets you are combining must look the same in order for you to be able to do this (ie. you must have the same number of columns and same column datatypes).

You have likely noticed the '(ALL)' added to this keyword in the title.  This is because the UNION operator comes in two flavours; UNION, and UNION ALL.

UNION will stack two datasets on top of one another but also de-dupes the output.

UNION ALL will stack two datasets on top of one another unaltered, therefore can include duplicates.

Visually it looks as follows:

![Union](https://www.sqlworld.co.uk/images/Blogs/SQLServer/Union.jpg)

Referring back to our two salesOrderHeader tables we can see that they contain slightly different data and, crucially, a different number of columns (Small2 does not contain deliveryDate).

Therefore in order to utilise UNION we will use a column list:

In [ ]:
select orderID, orderDate, salesOrderNo, customerID, employeeID, totalDue
from JupyterDatabase.sales.salesOrderHeaderSmall
union all
select orderID, orderDate, salesOrderNo, customerID, employeeID, totalDue
from JupyterDatabase.sales.salesOrderHeaderSmall2;

There are 10 records in sales.salesOrderHeaderSmall and 8 in sales.salesOrderHeaderSmall2. Therefore the above query will return us 18 records.

With only 18 records in our result set it is also easy to spot the duplicates created with UNION ALL.

Running the same again but this time with UNION on its own we can see we get a similar style of result but with no duplication:

In [ ]:
select orderID, orderDate, salesOrderNo, customerID, employeeID, totalDue
from JupyterDatabase.sales.salesOrderHeaderSmall
union
select orderID, orderDate, salesOrderNo, customerID, employeeID, totalDue
from JupyterDatabase.sales.salesOrderHeaderSmall2;

#### EXCEPT

This is a very powerful feature which allows for the comparison of two tables rather than their combination.

It can be crucial when troubleshooting or amending code to track any changes in output that have been caused by code changes or new logic.

The purpose of the keyword is to show us any records which are in the top dataset but not in the second dataset:

![Except](https://www.sqlworld.co.uk/images/Blogs/SQLServer/Except.jpg)

For this example we shall presume that sales.salesOrderHeaderSmall is our original dataset that we were happy with and sales.salesOrderHeaderSmall2 is the dataset we obtained following some logic changes.

We wish to know what, if anything, has changed as a result of our changes. To do this we can run the EXCEPT statement twice (once to find anything in sales.salesOrderHeaderSmall not in sales.salesOrderHeaderSmall2 and once to find anything in sales.salesOrderHeaderSmall2 not in sales.salesOrderHeaderSmall):

In [ ]:
select orderID, orderDate, salesOrderNo, customerID, employeeID, totalDue
from sales.salesOrderHeaderSmall
except
select orderID, orderDate, salesOrderNo, customerID, employeeID, totalDue
from sales.salesOrderHeaderSmall2;

select orderID, orderDate, salesOrderNo, customerID, employeeID, totalDue
from sales.salesOrderHeaderSmall2
except
select orderID, orderDate, salesOrderNo, customerID, employeeID, totalDue
from sales.salesOrderHeaderSmall;

We can see from the above that our code changes have resulted in several changes. There are three records in sales.salesOrderHeaderSmall which are no longer in sales.salesOrderHeaderSmall2, and there is also a record in sales.salesOrderHeaderSmall2 which is not in the original sales.salesOrderHeaderSmall table.

This is a task that would otherwise be very cumbersome to achieve.

#### INTERSECT

INTERSECT is similar to EXCEPT in that it is a comparison of two datasets rather than a combination.  However, this keyword will, instead of differences, return anything that is in both sets of data:

![Intersect](https://www.sqlworld.co.uk/images/Blogs/SQLServer/Intersect.jpg)

Again, we need to ensure that we are comparing the same number of columns and datatypes, therefore we will once again provide a column list:

In [ ]:
select orderID, orderDate, salesOrderNo, customerID, employeeID, totalDue
from sales.salesOrderHeaderSmall
intersect
select orderID, orderDate, salesOrderNo, customerID, employeeID, totalDue
from sales.salesOrderHeaderSmall2;

Hence there are seven identical records across the two tables.

#### Summary

Earlier we had three questions posed:

-   Query One
    -   I wish to see all orderIDs that have been placed without duplication
-   Query Two
    -   Which employee IDs have made a sale in both tables
-   Query Three
    -   Are there any Order Dates which are in sales.salesOrderHeaderSmall but not in sales.salesOrderHeaderSmall2

So let's revisit these and produce the relevant queries using the keywords we have just encountered.

<u>Query One</u>

I wish to see all orderIDs that have been placed without duplication.

This implies we want to see all orderIDs across both of our sales.salesOrderHeaderSmall tables and therefore we would be looking towards UNION and UNION ALL.

However, it also specifies that we require no duplication and therefore we are immediately thinking UNION and that we only need select OrderID to fulfil the requirement.

Hence our query is simply:

In [ ]:
select orderID
from JupyterDatabase.sales.salesOrderHeaderSmall
union
select orderID
from JupyterDatabase.sales.salesOrderHeaderSmall2;

<u>Query Two</u>

Which employee IDs have made a sale in both tables.

In order to find any records which are in both of two datasets we need to use INTERSECT:

In [ ]:
select employeeID
from JupyterDatabase.sales.salesOrderHeaderSmall
intersect
select employeeID
from JupyterDatabase.sales.salesOrderHeaderSmall2;

<u>Query Three</u>

Are there any Order Dates which as in sales.salesOrderHeaderSmall but not in sales.salesOrderHeaderSmall2?

If we wish to see differences between datasets (ie. Order Dates in one table but not another) then we can utilise the EXCEPT keyword.

As we are looking for just records in sales.salesOrderHeaderSmall that are not in sales.salesOrderHeaderSmall2 and not differences in general we can do this in a single statement:

In [ ]:
select orderDate
from sales.salesOrderHeaderSmall
except
select orderDate
from sales.salesOrderHeaderSmall2;